In [1]:
import pandas as pd
import numpy as np
from matplotlib.pyplot import subplots

In [2]:
# load in dataset
main = pd.read_csv("../data/unprocessed/wildfire-dataset_copy.csv", low_memory=False)

In [3]:
# All columns
all_columns = list(main.columns)
print(all_columns)

['FOD_ID', 'FPA_ID', 'SOURCE_SYSTEM_TYPE', 'SOURCE_SYSTEM', 'NWCG_REPORTING_AGENCY', 'NWCG_REPORTING_UNIT_ID', 'NWCG_REPORTING_UNIT_NAME', 'SOURCE_REPORTING_UNIT', 'SOURCE_REPORTING_UNIT_NAME', 'LOCAL_FIRE_REPORT_ID', 'LOCAL_INCIDENT_ID', 'FIRE_CODE', 'FIRE_NAME', 'ICS_209_PLUS_INCIDENT_JOIN_ID', 'ICS_209_PLUS_COMPLEX_JOIN_ID', 'MTBS_ID', 'MTBS_FIRE_NAME', 'COMPLEX_NAME', 'FIRE_YEAR', 'DISCOVERY_DATE', 'DISCOVERY_DOY', 'DISCOVERY_TIME', 'NWCG_CAUSE_CLASSIFICATION', 'NWCG_GENERAL_CAUSE', 'NWCG_CAUSE_AGE_CATEGORY', 'CONT_DATE', 'CONT_DOY', 'CONT_TIME', 'FIRE_SIZE', 'FIRE_SIZE_CLASS', 'LATITUDE', 'LONGITUDE', 'OWNER_DESCR', 'STATE', 'COUNTY', 'FIPS_CODE', 'FIPS_NAME']


In [4]:
# plan to drop columns
dropped_columns = [
    "ICS_209_PLUS_INCIDENT_JOIN_ID", 
    "ICS_209_PLUS_COMPLEX_JOIN_ID", 
    "MTBS_ID", 
    "MTBS_FIRE_NAME", 
    "COMPLEX_NAME", 
    "LOCAL_FIRE_REPORT_ID",
    "FIRE_CODE", 
    "FIRE_NAME", 
    "LOCAL_INCIDENT_ID", 
    "NWCG_CAUSE_AGE_CATEGORY", 
    "CONT_DATE", 
    "CONT_DOY", 
    "CONT_TIME", 
    "FIPS_CODE", 
    "FIPS_NAME", 
    "SOURCE_SYSTEM_TYPE", 
    "SOURCE_SYSTEM", 
    "NWCG_REPORTING_AGENCY",
    "NWCG_REPORTING_UNIT_ID",
    "NWCG_REPORTING_UNIT_NAME",
    "SOURCE_REPORTING_UNIT",
    "SOURCE_REPORTING_UNIT_NAME",
    "OWNER_DESCR",
    "FPA_ID",
    ]

In [5]:
# Creating a new list of columns
# Columns in all_columns but not in dropped_columns

columns = list(set(all_columns) - set(dropped_columns))
print(columns)

['DISCOVERY_TIME', 'DISCOVERY_DATE', 'FIRE_YEAR', 'FIRE_SIZE', 'COUNTY', 'NWCG_CAUSE_CLASSIFICATION', 'LATITUDE', 'STATE', 'DISCOVERY_DOY', 'LONGITUDE', 'FIRE_SIZE_CLASS', 'FOD_ID', 'NWCG_GENERAL_CAUSE']


In [6]:
# using the subset of columns to make a new dataframe to use
main_dataframe = main[columns]
main_dataframe.head()

,DISCOVERY_TIME,DISCOVERY_DATE,FIRE_YEAR,FIRE_SIZE,COUNTY,NWCG_CAUSE_CLASSIFICATION,LATITUDE,STATE,DISCOVERY_DOY,LONGITUDE,FIRE_SIZE_CLASS,FOD_ID,NWCG_GENERAL_CAUSE
0,1300.0,2/2/2005 0:00,2005,0.10,63,Human,40.036944,CA,33,-121.005833,A,1,Power generation/transmission/distribution
1,845.0,5/12/2004 0:00,2004,0.25,61,Natural,38.933056,CA,133,-120.404444,A,2,Natural
2,1921.0,5/31/2004 0:00,2004,0.10,17,Human,38.984167,CA,152,-120.735556,A,3,Debris and open burning
3,1600.0,6/28/2004 0:00,2004,0.10,3,Natural,38.559167,CA,180,-119.913333,A,4,Natural
4,1600.0,6/28/2004 0:00,2004,0.10,3,Natural,38.559167,CA,180,-119.933056,A,5,Natural


In [7]:
# Checking for missing values

main_dataframe.isna().sum()

DISCOVERY_TIME               754468
DISCOVERY_DATE                    0
FIRE_YEAR                         0
FIRE_SIZE                         0
COUNTY                       657235
NWCG_CAUSE_CLASSIFICATION         1
LATITUDE                          0
STATE                             0
DISCOVERY_DOY                     0
LONGITUDE                         0
FIRE_SIZE_CLASS                   0
FOD_ID                            0
NWCG_GENERAL_CAUSE                0
dtype: int64

In [8]:
# Using the coordinates to LATITUDE and LONGITUDE to find the county

import reverse_geocoder as rg

df = main_dataframe.copy()

# empty out all data in "county" field
df["COUNTY"] = None

# list all missing counties
missing_counties = df[df["COUNTY"].isna()].copy()

# get coordinates for all missing counties
coordinates = list(zip(missing_counties.LATITUDE, missing_counties.LONGITUDE))

# get data for missing counties
counties = rg.search(coordinates)

# get county names from counties dictionary
county_names =  [county["admin2"] for county in counties]

# check first 3 missing counties 
print(f"County Name: {county_names[:3]}")

# replace all missing counties with their corresponding counties
df.loc[df["COUNTY"].isna(), "COUNTY"] = county_names

# I have decided to make all current county codes into None values then fill them with their respective county names.

Loading formatted geocoded file...
County Name: ['Plumas County', 'El Dorado County', 'Placer County']


In [9]:
# Have decided to scrap this for the time being, will be converting all counties in to their respective name then encoding them.
# will convert county name to county code
# https://www2.census.gov/geo/docs/reference/codes2020/national_cousub2020.txt

# state_data = pd.read_csv("../data/unprocessed/national_cousub2020.txt", delimiter="|", low_memory=False)

# # Create a mapping DataFrame
# county_code_map = state_data[['COUNTYNAME', 'COUNTYFP']].drop_duplicates()

# # Merge main_dataframe with county_code_map
# main_dataframe = pd.merge(main_dataframe, county_code_map, left_on='COUNTY', right_on='COUNTYNAME', how='left')

# # Rename the column as needed and drop the extra column
# main_dataframe.rename(columns={'COUNTYFP': 'COUNTY_CODE'}, inplace=True)
# main_dataframe.drop('COUNTYNAME', axis=1, inplace=True)

# Now main_dataframe should have a new column 'COUNTY_CODE' with the corresponding county codes

In [10]:
df.head()

,DISCOVERY_TIME,DISCOVERY_DATE,FIRE_YEAR,FIRE_SIZE,COUNTY,NWCG_CAUSE_CLASSIFICATION,LATITUDE,STATE,DISCOVERY_DOY,LONGITUDE,FIRE_SIZE_CLASS,FOD_ID,NWCG_GENERAL_CAUSE
0,1300.0,2/2/2005 0:00,2005,0.10,Plumas County,Human,40.036944,CA,33,-121.005833,A,1,Power generation/transmission/distribution
1,845.0,5/12/2004 0:00,2004,0.25,El Dorado County,Natural,38.933056,CA,133,-120.404444,A,2,Natural
2,1921.0,5/31/2004 0:00,2004,0.10,Placer County,Human,38.984167,CA,152,-120.735556,A,3,Debris and open burning
3,1600.0,6/28/2004 0:00,2004,0.10,Douglas County,Natural,38.559167,CA,180,-119.913333,A,4,Natural
4,1600.0,6/28/2004 0:00,2004,0.10,El Dorado County,Natural,38.559167,CA,180,-119.933056,A,5,Natural


In [11]:
# splitting up Discovery date
# We already have the year, therefore we only need the month and day from this field.

# First we convert this column into datetime datatype
df["DISCOVERY_DATE"] = pd.to_datetime(df["DISCOVERY_DATE"])

# Now we extract the Month and Day
df["FIRE_MONTH"] = df["DISCOVERY_DATE"].dt.month
df["FIRE_DAY"] = df["DISCOVERY_DATE"].dt.day

# Now we verify change
df.head()

,DISCOVERY_TIME,DISCOVERY_DATE,FIRE_YEAR,FIRE_SIZE,COUNTY,NWCG_CAUSE_CLASSIFICATION,LATITUDE,STATE,DISCOVERY_DOY,LONGITUDE,FIRE_SIZE_CLASS,FOD_ID,NWCG_GENERAL_CAUSE,FIRE_MONTH,FIRE_DAY
0,1300.0,2005-02-02,2005,0.10,Plumas County,Human,40.036944,CA,33,-121.005833,A,1,Power generation/transmission/distribution,2,2
1,845.0,2004-05-12,2004,0.25,El Dorado County,Natural,38.933056,CA,133,-120.404444,A,2,Natural,5,12
2,1921.0,2004-05-31,2004,0.10,Placer County,Human,38.984167,CA,152,-120.735556,A,3,Debris and open burning,5,31
3,1600.0,2004-06-28,2004,0.10,Douglas County,Natural,38.559167,CA,180,-119.913333,A,4,Natural,6,28
4,1600.0,2004-06-28,2004,0.10,El Dorado County,Natural,38.559167,CA,180,-119.933056,A,5,Natural,6,28


In [12]:
# Since we split up discovery date we no longer need the field, so we drop it.
df = df.drop(columns=["DISCOVERY_DATE"], axis=1)

# verify change
df.head()

,DISCOVERY_TIME,FIRE_YEAR,FIRE_SIZE,COUNTY,NWCG_CAUSE_CLASSIFICATION,LATITUDE,STATE,DISCOVERY_DOY,LONGITUDE,FIRE_SIZE_CLASS,FOD_ID,NWCG_GENERAL_CAUSE,FIRE_MONTH,FIRE_DAY
0,1300.0,2005,0.10,Plumas County,Human,40.036944,CA,33,-121.005833,A,1,Power generation/transmission/distribution,2,2
1,845.0,2004,0.25,El Dorado County,Natural,38.933056,CA,133,-120.404444,A,2,Natural,5,12
2,1921.0,2004,0.10,Placer County,Human,38.984167,CA,152,-120.735556,A,3,Debris and open burning,5,31
3,1600.0,2004,0.10,Douglas County,Natural,38.559167,CA,180,-119.913333,A,4,Natural,6,28
4,1600.0,2004,0.10,El Dorado County,Natural,38.559167,CA,180,-119.933056,A,5,Natural,6,28


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2166753 entries, 0 to 2166752
Data columns (total 14 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   DISCOVERY_TIME             float64
 1   FIRE_YEAR                  int64  
 2   FIRE_SIZE                  float64
 3   COUNTY                     object 
 4   NWCG_CAUSE_CLASSIFICATION  object 
 5   LATITUDE                   float64
 6   STATE                      object 
 7   DISCOVERY_DOY              int64  
 8   LONGITUDE                  float64
 9   FIRE_SIZE_CLASS            object 
 10  FOD_ID                     int64  
 11  NWCG_GENERAL_CAUSE         object 
 12  FIRE_MONTH                 int32  
 13  FIRE_DAY                   int32  
dtypes: float64(4), int32(2), int64(3), object(5)
memory usage: 214.9+ MB


In [14]:
df.isna().sum()

DISCOVERY_TIME               754468
FIRE_YEAR                         0
FIRE_SIZE                         0
COUNTY                            0
NWCG_CAUSE_CLASSIFICATION         1
LATITUDE                          0
STATE                             0
DISCOVERY_DOY                     0
LONGITUDE                         0
FIRE_SIZE_CLASS                   0
FOD_ID                            0
NWCG_GENERAL_CAUSE                0
FIRE_MONTH                        0
FIRE_DAY                          0
dtype: int64

In [15]:
# We will convert discovery time into time as cyclical showing closeness between 23:59 and 0:00
# Issue: Deal with NaN floats, aka missing values

# get time mean
time_mean = df["DISCOVERY_TIME"].mean()

# fill in missing values with the mean 
df.fillna({"DISCOVERY_TIME": time_mean}, inplace=True)

# A function to extract the hour and minutes from discovery time.
def extract_time(time):
    hour = int(time // 100)
    minute = int(time % 100)
    return hour, minute

df["FIRE_HOUR"], df["FIRE_MINUTE"] = zip(*df["DISCOVERY_TIME"].apply(extract_time))

df["FIRE_HOUR_SIN"] = np.sin(2 * np.pi * df["FIRE_HOUR"]/24)
df["FIRE_HOUR_COS"] = np.cos(2 * np.pi * df["FIRE_HOUR"]/24)
df["FIRE_MINUTE_SIN"] = np.sin(2 * np.pi * df["FIRE_MINUTE"]/60)
df["FIRE_MINUTE_COS"] = np.cos(2 * np.pi * df["FIRE_MINUTE"]/60)


In [16]:
# verify time change
df.head()

,DISCOVERY_TIME,FIRE_YEAR,FIRE_SIZE,COUNTY,NWCG_CAUSE_CLASSIFICATION,LATITUDE,STATE,DISCOVERY_DOY,LONGITUDE,FIRE_SIZE_CLASS,FOD_ID,NWCG_GENERAL_CAUSE,FIRE_MONTH,FIRE_DAY,FIRE_HOUR,FIRE_MINUTE,FIRE_HOUR_SIN,FIRE_HOUR_COS,FIRE_MINUTE_SIN,FIRE_MINUTE_COS
0,1300.0,2005,0.10,Plumas County,Human,40.036944,CA,33,-121.005833,A,1,Power generation/transmission/distribution,2,2,13,0,-0.258819,-0.965926,0.000000,1.000000e+00
1,845.0,2004,0.25,El Dorado County,Natural,38.933056,CA,133,-120.404444,A,2,Natural,5,12,8,45,0.866025,-0.500000,-1.000000,-1.836970e-16
2,1921.0,2004,0.10,Placer County,Human,38.984167,CA,152,-120.735556,A,3,Debris and open burning,5,31,19,21,-0.965926,0.258819,0.809017,-5.877853e-01
3,1600.0,2004,0.10,Douglas County,Natural,38.559167,CA,180,-119.913333,A,4,Natural,6,28,16,0,-0.866025,-0.500000,0.000000,1.000000e+00
4,1600.0,2004,0.10,El Dorado County,Natural,38.559167,CA,180,-119.933056,A,5,Natural,6,28,16,0,-0.866025,-0.500000,0.000000,1.000000e+00


In [17]:
# Now we can drop the field discovery_time

df = df.drop(columns=["DISCOVERY_TIME"], axis=1)

In [18]:
df.isna().sum()

FIRE_YEAR                    0
FIRE_SIZE                    0
COUNTY                       0
NWCG_CAUSE_CLASSIFICATION    1
LATITUDE                     0
STATE                        0
DISCOVERY_DOY                0
LONGITUDE                    0
FIRE_SIZE_CLASS              0
FOD_ID                       0
NWCG_GENERAL_CAUSE           0
FIRE_MONTH                   0
FIRE_DAY                     0
FIRE_HOUR                    0
FIRE_MINUTE                  0
FIRE_HOUR_SIN                0
FIRE_HOUR_COS                0
FIRE_MINUTE_SIN              0
FIRE_MINUTE_COS              0
dtype: int64

In [19]:
# We will drop the record where their is a missing value in the following field, then check for any more missing values.
df = df.dropna(subset=["NWCG_CAUSE_CLASSIFICATION"])
df.isna().sum()

FIRE_YEAR                    0
FIRE_SIZE                    0
COUNTY                       0
NWCG_CAUSE_CLASSIFICATION    0
LATITUDE                     0
STATE                        0
DISCOVERY_DOY                0
LONGITUDE                    0
FIRE_SIZE_CLASS              0
FOD_ID                       0
NWCG_GENERAL_CAUSE           0
FIRE_MONTH                   0
FIRE_DAY                     0
FIRE_HOUR                    0
FIRE_MINUTE                  0
FIRE_HOUR_SIN                0
FIRE_HOUR_COS                0
FIRE_MINUTE_SIN              0
FIRE_MINUTE_COS              0
dtype: int64

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2166752 entries, 0 to 2166752
Data columns (total 19 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   FIRE_YEAR                  int64  
 1   FIRE_SIZE                  float64
 2   COUNTY                     object 
 3   NWCG_CAUSE_CLASSIFICATION  object 
 4   LATITUDE                   float64
 5   STATE                      object 
 6   DISCOVERY_DOY              int64  
 7   LONGITUDE                  float64
 8   FIRE_SIZE_CLASS            object 
 9   FOD_ID                     int64  
 10  NWCG_GENERAL_CAUSE         object 
 11  FIRE_MONTH                 int32  
 12  FIRE_DAY                   int32  
 13  FIRE_HOUR                  int64  
 14  FIRE_MINUTE                int64  
 15  FIRE_HOUR_SIN              float64
 16  FIRE_HOUR_COS              float64
 17  FIRE_MINUTE_SIN            float64
 18  FIRE_MINUTE_COS            float64
dtypes: float64(7), int32(2), int64(5), object(5)
me

In [21]:
# TODO: Convert all objects into numerical values.

First we will begin by looking at how the data looks like for each column that has object data type.

In [22]:
df["NWCG_CAUSE_CLASSIFICATION"].unique()

array(['Human', 'Natural', 'Missing data/not specified/undetermined'],
      dtype=object)

In [23]:
df["FIRE_SIZE_CLASS"].unique()

array(['A', 'B', 'G', 'C', 'D', 'F', 'E'], dtype=object)

In [24]:
df["STATE"].unique()

array(['CA', 'NM', 'OR', 'NC', 'WY', 'CO', 'WA', 'MT', 'UT', 'AZ', 'SD',
       'AR', 'NV', 'ID', 'MN', 'TX', 'FL', 'SC', 'LA', 'OK', 'KS', 'MO',
       'NE', 'MI', 'KY', 'OH', 'IN', 'VA', 'IL', 'TN', 'GA', 'AK', 'ND',
       'WV', 'WI', 'AL', 'NH', 'PA', 'MS', 'ME', 'VT', 'NY', 'IA', 'DC',
       'MD', 'CT', 'MA', 'NJ', 'HI', 'DE', 'PR', 'RI'], dtype=object)

In [25]:
df["NWCG_GENERAL_CAUSE"].unique()

array(['Power generation/transmission/distribution', 'Natural',
       'Debris and open burning',
       'Missing data/not specified/undetermined',
       'Recreation and ceremony', 'Equipment and vehicle use',
       'Arson/incendiarism', 'Fireworks', 'Other causes',
       'Railroad operations and maintenance', 'Smoking',
       'Misuse of fire by a minor', 'Firearms and explosives use'],
      dtype=object)

In [26]:
df["COUNTY"].unique()

array(['Plumas County', 'El Dorado County', 'Placer County', ...,
       'Kenosha County', 'Pepin County', 'Moultrie County'], dtype=object)

Let the encoding begin, I will be merging the column "STATE" and "COUNTY" because their may be some counties that have the same name across different states. 

After that I will use One hot encoder on the following columns, "NWCG_CAUSE_CLASSIFICATION", "NWCG_GENERAL_CAUSE", and "STATE_COUNTY".

Since their is no hierarchy in these three columns one hot encoding will be a good choice. 

Then I will apply ordinal encoding to the column "FIRE_SIZE_CLASS" because I would like to show an order between A - G

In [27]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2166752 entries, 0 to 2166752
Data columns (total 19 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   FIRE_YEAR                  int64  
 1   FIRE_SIZE                  float64
 2   COUNTY                     object 
 3   NWCG_CAUSE_CLASSIFICATION  object 
 4   LATITUDE                   float64
 5   STATE                      object 
 6   DISCOVERY_DOY              int64  
 7   LONGITUDE                  float64
 8   FIRE_SIZE_CLASS            object 
 9   FOD_ID                     int64  
 10  NWCG_GENERAL_CAUSE         object 
 11  FIRE_MONTH                 int32  
 12  FIRE_DAY                   int32  
 13  FIRE_HOUR                  int64  
 14  FIRE_MINUTE                int64  
 15  FIRE_HOUR_SIN              float64
 16  FIRE_HOUR_COS              float64
 17  FIRE_MINUTE_SIN            float64
 18  FIRE_MINUTE_COS            float64
dtypes: float64(7), int32(2), int64(5), object(5)
me

In [28]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# concatenate state and county
df["STATE_COUNTY"] = df["STATE"] + "-" + df["COUNTY"]

# selecting columns that we will encode for onehotencoding
columns_to_encode_oh = ["NWCG_CAUSE_CLASSIFICATION", "NWCG_GENERAL_CAUSE", "STATE_COUNTY"]

# Instatiate OneHotEncoder 
oh_encoder = OneHotEncoder()

# Fit and transform 
oh_encoded = oh_encoder.fit_transform(df[columns_to_encode_oh])


In [29]:
# now encoding fire_size_class with ordinal
ordinal_encoder = OrdinalEncoder(categories=[["A", "B", "C", "D", "E", "F", "G"]])


In [30]:

df["FIRE_SIZE_CLASS_ENCODED"] = ordinal_encoder.fit_transform(df[["FIRE_SIZE_CLASS"]]) 

In [31]:
# creating a df with the encoded data 
df_encoded = pd.DataFrame.sparse.from_spmatrix(oh_encoded, columns=oh_encoder.get_feature_names_out())

In [32]:
# Later on we can revert the dataset into a dense format for further data analysis here is the code:
# df_dense = df_encoded.sparse.to_dense()

In [33]:
df = df.drop(columns=columns_to_encode_oh + ["FIRE_SIZE_CLASS"])

final_df = pd.concat([df, df_encoded, df[["FIRE_SIZE_CLASS_ENCODED"]]], axis=1)

In [34]:
final_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 2166753 entries, 0 to 2166705
Data columns (total 3803 columns):
 #     Column                                                             Dtype             
---    ------                                                             -----             
 0     FIRE_YEAR                                                          float64           
 1     FIRE_SIZE                                                          float64           
 2     COUNTY                                                             object            
 3     LATITUDE                                                           float64           
 4     STATE                                                              object            
 5     DISCOVERY_DOY                                                      float64           
 6     LONGITUDE                                                          float64           
 7     FOD_ID                                        

In [35]:
final_df = final_df.drop(columns=["STATE", "COUNTY"], axis=1)

In [36]:
final_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 2166753 entries, 0 to 2166705
Data columns (total 3801 columns):
 #     Column                                                             Dtype             
---    ------                                                             -----             
 0     FIRE_YEAR                                                          float64           
 1     FIRE_SIZE                                                          float64           
 2     LATITUDE                                                           float64           
 3     DISCOVERY_DOY                                                      float64           
 4     LONGITUDE                                                          float64           
 5     FOD_ID                                                             float64           
 6     FIRE_MONTH                                                         float64           
 7     FIRE_DAY                                      

will save file as a parquet, make sure to have pyarrow and fastparquet as your dependencies already

In [37]:
# TODO: save as csv's 

final_df.to_parquet("../data/processed/encoded_data.parquet")

ValueError: Duplicate column names found: ['FIRE_YEAR', 'FIRE_SIZE', 'LATITUDE', 'DISCOVERY_DOY', 'LONGITUDE', 'FOD_ID', 'FIRE_MONTH', 'FIRE_DAY', 'FIRE_HOUR', 'FIRE_MINUTE', 'FIRE_HOUR_SIN', 'FIRE_HOUR_COS', 'FIRE_MINUTE_SIN', 'FIRE_MINUTE_COS', 'FIRE_SIZE_CLASS_ENCODED', 'NWCG_CAUSE_CLASSIFICATION_Human', 'NWCG_CAUSE_CLASSIFICATION_Missing data/not specified/undetermined', 'NWCG_CAUSE_CLASSIFICATION_Natural', 'NWCG_GENERAL_CAUSE_Arson/incendiarism', 'NWCG_GENERAL_CAUSE_Debris and open burning', 'NWCG_GENERAL_CAUSE_Equipment and vehicle use', 'NWCG_GENERAL_CAUSE_Firearms and explosives use', 'NWCG_GENERAL_CAUSE_Fireworks', 'NWCG_GENERAL_CAUSE_Missing data/not specified/undetermined', 'NWCG_GENERAL_CAUSE_Misuse of fire by a minor', 'NWCG_GENERAL_CAUSE_Natural', 'NWCG_GENERAL_CAUSE_Other causes', 'NWCG_GENERAL_CAUSE_Power generation/transmission/distribution', 'NWCG_GENERAL_CAUSE_Railroad operations and maintenance', 'NWCG_GENERAL_CAUSE_Recreation and ceremony', 'NWCG_GENERAL_CAUSE_Smok

In [38]:
duplicates = final_df.columns[final_df.columns.duplicated()]
print(f"Duplicate column names: {duplicates}")

Duplicate column names: Index(['FIRE_SIZE_CLASS_ENCODED'], dtype='object')


In [39]:
final_df = final_df.drop_duplicates()

duplicates = final_df.columns[final_df.columns.duplicated()]
print(f"Duplicate column names: {duplicates}")

MemoryError: Unable to allocate 2.07 MiB for an array with shape (2166753,) and data type bool

In [ ]:
# TODO: Save Encoders